<a href="https://colab.research.google.com/github/Freddy-94/Metaheuristics-practices/blob/main/Simulated_annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Alfredo Bernal Luna
### Practice No. 1 "Simulated Annealing Algorithm".



# Definition of objective function:

Let $f : [0, \pi] ^n \to \mathbb{R}$ given by:

\begin{align}
        f(\mathbf{x}) = -\sum_{i=1}^n sin(x_{i})sin^{20}(i(\frac{x_{i}^{2}}{\pi})) 
\end{align}

In [ ]:
import numpy as np
import math 

def obj_fun(n, x):
  sum = 0
  for i in range(1, n+1):
    sum += (math.sin(x[i-1])) * (math.sin(i*(x[i-1]**2/math.pi))**20)
  fun_value = -sum
  return fun_value

# Test of the objective function:

In [ ]:
obj_fun(2, (0.26, 2.61))

-0.11940677899096029

# "Cooling" functions for the algorithm:

**1.   Linear cooling:**

$ T_{k} = T_{k-1} - \beta $

In [24]:
def linear(iteration, last_tmp, beta):
  for iter in range(iteration):
    last_tmp = last_tmp - beta
  new_tmp = last_tmp
  return new_tmp


**2.   Geometric cooling:**

$ T_{k} = \alpha * T_{k-1} $

In [25]:
def geometric(iteration, last_tmp, alpha):
  for iter in range(iteration):
    last_tmp = alpha * last_tmp
  new_tmp = last_tmp
  return new_tmp

**3.   Logarithmic cooling:**

$ T_{k} = \frac{ln(k)}{ln(k+1)} * T_{k-1} $

In [27]:
def log_cool(iteration, last_tmp):
  for iter in range(iteration):
    last_tmp = (math.log(iter)/math.log(iter+1))*last_tmp
  new_tmp = last_tmp
  return new_tmp

**4.   Hybrid cooling:**

$ T_{k} = \begin{cases} 
      (\frac{k}{k+1}) * T_{k-1} & k \leq β \\
      \alpha * T_{k-1} & k > β 
   \end{cases}
$

In [28]:
def hybrid(iteration, alpha, beta, last_tmp):
  for iter in range(iteration):
    if iteration <= beta:
      last_tmp = (iteration/iteration+1)*last_tmp
    else:
      last_tmp = alpha * last_tmp
  new_tmp = last_tmp
  return new_tmp

**5.   Exponential cooling:**

$ T_{k} = \frac{T_{k-1}}{1 + \beta T_{k-1}} $

In [29]:
def exp(iteration, beta, last_tmp):
  for iter in range(iteration):
    last_tmp = last_tmp/(1+(beta*last_tmp))
  new_tmp = last_tmp
  return new_tmp